# GDB LE5 Bericht

### Welche Datenbank hast Du gewählt?
MongoDB 

### Warum hast Du diese gewählt?
Wir haben Mongo DB ausgewählt, weil es open source ist und gut strukturierte Testdaten anbietet. Wir haben vom MongoDB Cloud die Datenbank "mflix" für die Abfragen ausgesucht. Nach Recherche haben wir herausgefunden, dass MongoDB folgende Vorteile hat und dass es die beliebteste NoSQL Datenbank ist:
- Leistung -> Abfrageleistung in MongoDB ist viel schneller
- Einfachheit -> einfache Query Sprache
- Flexibilität -> sehr dynamische Schaltplanarchitektur für unstrukturierte Daten
- Skalierbarkeit -> erleichtert die Erhöhung der Speicherkapazität.

### Wie sehen die Daten aus?
In MongoDB werden Daten als Dokumente gespeichert. Diese Dokumente werden in MongoDB im JSON-Format (JavaScript Object Notation) gespeichert. JSON-Dokumente unterstützen eingebettete Felder, sodass verwandte Daten und Datenlisten mit dem Dokument anstelle einer externen Tabelle gespeichert werden können. Die Tabellen in MongoDB heissen Collections und haben keine Datenstrukturschema verglichen mit relationale Datenbanken.

### Inwiefern ist das besser als eine relationale Datenbank?
- Geeignet für hierarchische Datenspeicherung
- Es ist horizontal skalierbar, d. h. wir können weitere Server hinzufügen.
- Es hat ein dynamisches Schema.
- Es ist fast 100-mal schneller als relationale Datenbanken.
- Es unterstützt die JSON-Query zusammen mit SQL.

### Wie sehen komplexe Fragestellungen (Abfrage) zu den Daten aus, und warum sind sie komplex? Vergleiche es mit eine SQL, würde es komplizierter sein, welche Vorteile gibt es gegenüber SQL?
Bei einfachen Abfragen werden die Daten nur abgelesen, jedoch bei komplexen Abfragen werden sie gefiltert, verarbeitet und mit anderen Daten zusammengefügt (kombiniert)

### Unsere Abfragen
#### Abfrage 1
Was ist die durchschnittliche Bewertung aller Filme in der Schweiz?

#### Abfrage 2
Welche Filme wurden vor 1970 gedreht und haben mehr als 100 Kommentare?

#### Abfrage 3
Welche 10 Filme haben die meisten Kommentare?

#### Abfrage 4
Was sind die besten fünf Schweizer Filme in Bezug auf die Schweizer Durchschnittsbewertung?

#### Abfrage 5
Welche Genre hat die meisten Auszeichnungen (Awards)

#### Abfrage 6
Welcher Bundesstaat besitzt die meisten Theater?

### Code (Vorbereitung)
Zuerst müssen wir uns mit dem MongoDB Cloud verbinden und die gewünschte Datenbank "mflix" auswählen.

In [32]:
from pymongo import MongoClient
import urllib
import pandas
import configparser
import pprint

config = configparser.ConfigParser()
config.read('config.ini')

db_username = config["MongoDB"]["username"]
db_password = config["MongoDB"]["password"]
db_hostname = config["MongoDB"]["hostname"]
db_link = config["MongoDB"]["link"]

In [133]:
!curl ipecho.net/plain

147.86.207.49


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    13  100    13    0     0    100      0 --:--:-- --:--:-- --:--:--   100


In [36]:
# Verbindung mit Mongo Atlas
client = MongoClient('mongodb+srv://{}:{}@{}.{}.mongodb.net/sample_mflix'.format(db_username, db_password, db_hostname, db_link))
db = client.sample_mflix

In [37]:
# Collections der Datenbank anzeigen
db.list_collection_names()

['sessions', 'users', 'movies', 'theaters', 'comments']

### Frage 1
#### Was ist die durchschnittliche Bewertung aller Filme in der Schweiz?

In [72]:
pipeline = db.movies.aggregate([
    {
        '$match': {
            'countries': 'Switzerland'
        }
    }, {
        '$group': {
            '_id': {
                '$first': '$rating'
            }, 
            'averageRating': {
                '$avg': '$imdb.rating'
            }
        }
    }, {
        '$project': {
            'rating': 0, 
            '_id': 0
        }
    }
])

for var in pipeline:
    pprint.pprint(var)

{'averageRating': 6.890000000000001}


### Frage 2
#### Welche Filme wurden vor 1970 gedreht und haben mehr als 100 Kommentare?

In [98]:
pipeline = db.movies.aggregate([
    {
        '$addFields': {
            'movie_id': {
                '$toString': '$_id'
            }
        }
    }, {
        '$lookup': {
            'from': 'comments', 
            'localField': '_id', 
            'foreignField': 'movie_id', 
            'as': 'comments'
        }
    }, {
        '$match': {
            'year': {
                '$lt': 1970
            }, 
            'num_mflix_comments': {
                '$gt': 100
            }
        }
    }, {
        '$sort': {
            'num_mflix_comments': -1
        }
    }, {
        '$project': {
            '_id': 0, 
            'movie_title': '$title', 
            'comment_amount': '$num_mflix_comments'
        }
    }
])

for var in pipeline:
    pprint.pprint(var)

{'comment_amount': 153, 'movie_title': 'Cinderella'}
{'comment_amount': 139, 'movie_title': 'The Wizard of Oz'}
{'comment_amount': 134, 'movie_title': 'The Jungle Book'}
{'comment_amount': 128, 'movie_title': 'Mary Poppins'}
{'comment_amount': 126, 'movie_title': 'Lady and the Tramp'}
{'comment_amount': 118, 'movie_title': '101 Dalmatians'}


### Frage 3
#### Welche 10 Filme haben die meisten Kommentare?

In [81]:
pipeline = db.comments.aggregate([
    {
        '$group': {
            '_id': {
                'movie_id': '$movie_id'
            }, 
            'commentAmount': {
                '$sum': 1
            }
        }
    }, {
        '$match': {
            'commentAmount': {
                '$gt': 1
            }
        }
    }, {
        '$sort': {
            'commentAmount': -1
        }
    }, {
        '$lookup': {
            'from': 'movies', 
            'localField': '_id.movie_id', 
            'foreignField': '_id', 
            'as': 'movie'
        }
    }, {
        '$addFields': {
            'title': {
                '$first': '$movie.title'
            }
        }
    }, {
        '$limit': 10
    }, {
        '$project': {
            '_id': 0, 
            'title': 1, 
            'commentAmount': 1
        }
    }
])

for var in pipeline:
    pprint.pprint(var)

{'commentAmount': 161, 'title': 'The Taking of Pelham 1 2 3'}
{'commentAmount': 158, 'title': 'Terminator Salvation'}
{'commentAmount': 158, 'title': 'About a Boy'}
{'commentAmount': 158, 'title': '50 First Dates'}
{'commentAmount': 158, 'title': "Ocean's Eleven"}
{'commentAmount': 157, 'title': 'The Mummy'}
{'commentAmount': 157, 'title': 'Sherlock Holmes'}
{'commentAmount': 155, 'title': 'Hellboy II: The Golden Army'}
{'commentAmount': 154, 'title': 'The Mummy Returns'}
{'commentAmount': 154, 'title': 'Anchorman: The Legend of Ron Burgundy'}


### Frage 4
#### Was sind die besten fünf Schweizer Filme in Bezug auf die Schweizer Durchschnittsbewertung?

In [117]:
pipeline1 = db.movies.aggregate([
    {
        '$match': {
            'countries': 'Switzerland', 
            'imdb.rating': {
                '$gt': 0
            }
        }
    }, {
        '$addFields': {
            'movie_id': '$_id'
        }
    }, {
        '$group': {
            '_id': None, 
            'avgRating': {
                '$avg': '$imdb.rating'
            }, 
            'id': {
                '$addToSet': '$movie_id'
            }
        }
    }, {
        '$unwind': {
            'path': '$id'
        }
    }, {
        '$lookup': {
            'from': 'movies', 
            'localField': 'id', 
            'foreignField': '_id', 
            'as': 'movie'
        }
    }, {
        '$unwind': {
            'path': '$movie'
        }
    }, {
        '$sort': {
            'movie.imdb.rating': -1
        }
    }, {
        '$limit': 5
    },{
        '$project': {
            '_id': 0,
            'averageRating': '$avgRating',
            'title': '$movie.title',
            'rating': '$movie.imdb.rating'
        }
    }
])

for var in pipeline1:
    pprint.pprint(var)

{'averageRating': 6.890000000000001,
 'rating': 8.8,
 'title': 'Shining Night: A Portrait of Composer Morten Lauridsen'}
{'averageRating': 6.890000000000001, 'rating': 8.5, 'title': 'Satantango'}
{'averageRating': 6.890000000000001,
 'rating': 8.4,
 'title': 'The Sorrow and the Pity'}
{'averageRating': 6.890000000000001,
 'rating': 8.3,
 'title': 'The Sorrow and the Pity'}
{'averageRating': 6.890000000000001, 'rating': 8.2, 'title': 'Yol'}


### Frage 5
#### Welches sind die Filme mit den meisten Auszeichnungen pro Genre und wie viele Auszeichnungen sind das total?

In [124]:
pipeline1 = db.movies.aggregate([
    {
        '$group': {
            '_id': {
                '$first': '$genres'
            }, 
            'movie_title': {
                '$first': '$title'
            }, 
            'most_awards': {
                '$max': '$awards.wins'
            }, 
            'avg_awards': {
                '$avg': '$awards.wins'
            }
        }
    }, {
        '$addFields': {
            'difference': {
                '$concat': [
                    {
                        '$toString': {
                            '$subtract': [
                                {
                                    '$multiply': [
                                        {
                                            '$divide': [
                                                '$most_awards', '$avg_awards'
                                            ]
                                        }, 100
                                    ]
                                }, 100
                            ]
                        }
                    }, '%'
                ]
            }
        }
    }, {
        '$sort': {
            'most_awards': -1
        }
    }, {
        '$project': {
            '_id': 0, 
            'genre': '$_id', 
            'movie': '$movie_title', 
            'awards': '$most_awards', 
            'average_awards_genre': '$avg_awards', 
            'difference_to_average': '$difference'
        }
    }
])

for var in pipeline1:
    pprint.pprint(var)

{'average_awards_genre': 8.416393442622951,
 'awards': 267,
 'difference_to_average': '3072.38%',
 'genre': 'Biography',
 'movie': 'Regeneration'}
{'average_awards_genre': 6.365384615384615,
 'awards': 231,
 'difference_to_average': '3529%',
 'genre': 'Sci-Fi',
 'movie': 'Rocketship X-M'}
{'average_awards_genre': 3.2322533673097924,
 'awards': 210,
 'difference_to_average': '6397.02%',
 'genre': 'Comedy',
 'movie': 'The Poor Little Rich Girl'}
{'average_awards_genre': 5.077719510555331,
 'awards': 185,
 'difference_to_average': '3543.37%',
 'genre': 'Drama',
 'movie': 'In the Land of the Head Hunters'}
{'average_awards_genre': 4.8954545454545455,
 'awards': 175,
 'difference_to_average': '3474.74%',
 'genre': 'Adventure',
 'movie': 'The Last of the Mohicans'}
{'average_awards_genre': 4.986496624156039,
 'awards': 172,
 'difference_to_average': '3349.32%',
 'genre': 'Crime',
 'movie': 'Traffic in Souls'}
{'average_awards_genre': 3.3172205438066467,
 'awards': 162,
 'difference_to_averag

### Frage 6
#### Welche 15 Städte haben die meisten Theater und in welchem Bundesstaat befinden sie sich?

In [130]:
pipeline = db.theaters.aggregate([
    {
        '$group': {
            '_id': '$location.address.state', 
            'state_amount': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'state_amount': -1
        }
    }, {
        '$lookup': {
            'from': 'theaters', 
            'localField': '_id', 
            'foreignField': 'location.address.state', 
            'as': 'theaters'
        }
    }, {
        '$unwind': {
            'path': '$theaters'
        }
    }, {
        '$group': {
            '_id': '$theaters.location.address.city', 
            'city_theaters': {
                '$sum': 1
            }, 
            'state': {
                '$first': '$theaters.location.address.state'
            }, 
            'state_theaters': {
                '$first': '$state_amount'
            }
        }
    }, {
        '$sort': {
            'city_theaters': -1
        }
    }, {
        '$limit': 15
    }, {
        '$project': {
            '_id': 0, 
            'state': 1, 
            'state_theaters': 1, 
            'city': '$_id', 
            'city_theaters': 1
        }
    }
])
    
for var in pipeline:
    pprint.pprint(var)

{'city': 'Las Vegas', 'city_theaters': 29, 'state': 'NV', 'state_theaters': 39}
{'city': 'Houston', 'city_theaters': 22, 'state': 'TX', 'state_theaters': 160}
{'city': 'San Antonio',
 'city_theaters': 14,
 'state': 'TX',
 'state_theaters': 160}
{'city': 'Orlando', 'city_theaters': 13, 'state': 'FL', 'state_theaters': 111}
{'city': 'Dallas', 'city_theaters': 12, 'state': 'TX', 'state_theaters': 160}
{'city': 'Los Angeles',
 'city_theaters': 12,
 'state': 'CA',
 'state_theaters': 169}
{'city': 'Atlanta', 'city_theaters': 10, 'state': 'GA', 'state_theaters': 45}
{'city': 'Jacksonville',
 'city_theaters': 9,
 'state': 'FL',
 'state_theaters': 111}
{'city': 'San Francisco',
 'city_theaters': 9,
 'state': 'CA',
 'state_theaters': 169}
{'city': 'Chicago', 'city_theaters': 8, 'state': 'IL', 'state_theaters': 70}
{'city': 'New York', 'city_theaters': 8, 'state': 'NY', 'state_theaters': 81}
{'city': 'Miami', 'city_theaters': 8, 'state': 'FL', 'state_theaters': 111}
{'city': 'San Diego', 'city_th